# Magnetic Contol program
**`author`**: `Kuan-Chia Chiu`  
**`update`**: `2018-12-12`  
**`package installed`**: `pyvisa`, `serial`, `ipywidgets`(*optional*), `plotly`(*optional*)


In [45]:
import visa
import time
import serial
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt


rm = visa.ResourceManager()
res = rm.list_resources()
vna_dict =[]
for x in range(0, len(res)):
    vna_dict.append(res[x])
    print("{}. ".format(x) + res[x] + ", \n")
print("\n")


0. ASRL4::INSTR, 

1. ASRL5::INSTR, 

2. ASRL6::INSTR, 

3. GPIB0::6::INSTR, 





In [46]:
KEPCO_address = widgets.ToggleButtons(
    options=vna_dict,
    description="Power Sup.",
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=[],
#     icons=['check'] * 3
)

In [47]:
LakeShore_address = widgets.Text()
accordion = widgets.Accordion(children=[KEPCO_address, LakeShore_address])
accordion.set_title(0, 'Power Supply (KEPCO)')
accordion.set_title(1, 'Gauss meter (LakeShore)')
accordion

Accordion(children=(ToggleButtons(button_style='success', description='Power Sup.', options=('ASRL4::INSTR', '…

In [48]:
ser = serial.Serial(LakeShore_address.value, 57600,bytesize=7, parity=serial.PARITY_ODD, stopbits=1)  # Lake Shore 425
LakeShore_address.value

'COM6'

## Current to Field curves

In [52]:
sweep_current = 30
sweep_pts = 1601
KEPCO = rm.open_resource(KEPCO_address.value)

KEPCO.timeout=10000
KEPCO.write("FUNC:MODE CURR")
KEPCO.write("volt 0.000;curr 0.000")
KEPCO.write("OUTPUT ON")
Field_data = []
Current_data = []
for i in range(sweep_pts):
    KEPCO.write("volt  20.000;curr {:.4f}".format(i*sweep_current/(sweep_pts-1)))
    time.sleep(1)
    Current = str(i*sweep_current/(sweep_pts-1))
    Current_data.append(Current)
    ser.write(b'RDGFIELD?\n')
    Field = ser.readline()
    Field_data.append(Field.decode('utf8'))
    time.sleep(0.1)

KEPCO.write("OUTPUT OFF")
KEPCO.write("volt 0.000;curr 0.000")
#ser.close()


(23, <StatusCode.success: 0>)

## Save Data

In [53]:
import os
import re
# Add datetime automatically and append with the file name arbitaryly.
with open('./20181212_30A_delay_1s.txt', 'w') as f:
    f.write('Cuurent\s(A)\tField\s(G)\n')
    for j in range(sweep_pts):
        f.write('{}\t{}'.format(Current_data[j],Field_data[j]))
f.close()

## Plot data

In [54]:
import plotly as py
from plotly.graph_objs import Scatter, Layout
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
trace = go.Scatter(x=Current_data, y=Field_data, name='Original data')
iplot([trace], filename='test')